# Aula 3 - Parte Prática - Problema de Controle

## Introdução

Nesse notebook iremos implementar o algoritmo de controle DQN. Esse será o primeiro algoritmo de Deep RL que veremos no curso, isto é, combinaremos redes neurais e programação dinâmica aproximada. A proposta geral do DQN é combinar a regra de atualização do Q-Learning com aproximadores de função não-lineares para se obter boa generalização sobre o conjunto de possíveis observações.

<img src="img/control.png" alt="Agent-Env Loop" style="width: 750px;"/>

Na 1a parte desse notebook, nosso objetivo será resolver o ambiente `CartPole-v1`. Já na 2a parte tentaremos encontrar uma solução para o ambiente `PongNoFrameskip-v4` da suite ALE-Atari disponível via OpenAI `gym`.


### Objetivos:

- Entender o papel da otimalidade de Bellman para algoritmos de controle
- Desenvolver intuição sobre o problema de exploração em RL
- Ter um primeiro contato com técnicas de treinamento de algoritmos de deep RL
- Familiarizar-se com a biblioteca de redes neurais dm-sonnet

### Instalação

É necessário rodar a célula abaixo apenas uma vez para instalar as dependências do notebook. **Atenção**: reinicie o kernel depois de rodar a célula abaixo.

In [ ]:
# atualizar pip
!pip install -U pip setuptools

# instalar OpenAI gym do branch master ===> IMPORTANTE!
!pip install -e git+https://github.com/openai/gym.git@master#egg=gym
!pip install "gym[atari]"

# instalar TensorFlow e APIs de alto nível
!pip install tensorflow tensorflow-probability dm-sonnet

### Imports

> Não se esqueça de executar os imports abaixo antes de prosseguir com o notebook.

In [ ]:
from collections import deque
from datetime import datetime
import os.path as osp
import time

import gym
from gym.wrappers import AtariPreprocessing, FrameStack, Monitor, TimeLimit
import numpy as np
import sonnet as snt
import tensorflow as tf
from tqdm.notebook import trange

from utils import logging
from utils.nn import initializers
from utils import replay
from utils import schedule
from utils import tf_utils


tf_utils.set_tf_allow_growth() # necessário apenas se você dispõe de GPU

## Ambiente - CartPole-v1

Como veremos no exercício-programa de hoje, é sempre uma boa ideia em aprendizado por reforço iniciar o estudo de um algoritmo por um problema simples e pequeno para o qual você poderá resolver em poucos minutos. Para isso, o ambiente do `CartPole-v1` é usualmente um dos primeiros problemas que um agente baseado em aprendizado por reforço deve ser capaz de resolver antes de tentar atacar problemas mais complexos.

In [ ]:
def make_envs(env_id):
    env = gym.make(env_id)
    eval_env = gym.vector.make(env_id, num_envs=20, asynchronous=True)
    test_env = gym.make(env_id)
    return env, eval_env, test_env

In [ ]:
env, eval_env, test_env = make_envs("CartPole-v1")

> Dica: se você não estiver familiarizado com esse ambiente ou precisar refrescar a memória, lembre-se de consultar a documentação disponível no site do OpenAI Gym [https://gym.openai.com/envs/#classic_control](https://gym.openai.com/envs/#classic_control) e também procure entender principalmente os detalhes sobre o espaço de estados e ações do ambiente usando os métodos `env.observation_space` e `env.action_space`.

## Deep Q-Learning

Como visto em aula o algoritmo `DQN` procura aproximar a função $Q(s, a)$ utilizando redes neurais treinadas por meio da otimização de uma função objetivo baseada na regra de atualização do Q-Learning. O algoritmo abaixo descreve de maneira geral o treinamento de um agente `DQN`.

<img src="img/dqn-algo.png" alt="Agent-Env Loop" style="width: 500px;"/>

Nessa seção desenvolveremos os componentes desse algoritmo passo a passo:

1. **Redes neurais (networks)**: inicialmente construiremos usando a biblioteca `dm-sonnet` a rede neural para a função $Q(s, a)$;
2. **Função objetivo (loss)**: uma vez definida a classe da função $Q(s, a)$, implementaremos a função objetivo utilizada no problema de "regressão" que o Q-Learning tenta resolver;
3. **Atualização (update)**: em seguida instanciaremos um otimizador baseado em gradientes que será responsável por minimizar a função objetivo previamente definida; e
4. **Política $\epsilon$-greedy**: por fim definiremos a política estocástica para exploração.

### Redes Neurais (networks)

Para representar funções $Q(s,a)$ utilizando redes neurais, temos em geral 2 opções de implementação:
1. Definir uma rede com entrada $(s, a)$ e saída um único número real: $Q_\phi : \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$; ou
2. definir como entrada apenas o estado $s$ e saída um vetor de tamanho $|\mathcal{A}|$: $Q_\phi : \mathcal{S} \rightarrow \mathbb{R}^{|\mathcal{A}|}$ .

Em geral, para algoritmos baseados no `DQN` é costume utilizar a 2a opção.

> **Observação**: note que $\phi \in \mathbb{R}^d$ com $d \ll |S|$, onde $\phi$ denota o conjunto de parâmetros (i.e., *kernels* e *biases*) da rede neural. Dessa forma, a rede deve extrair apenas informações essenciais sobre o estado para a predição do retorno esperado (como vimos na aula de predição).

<img src="img/conv-net.png" alt="Agent-Env Loop" style="width: 550px;"/>

In [ ]:
class QNetwork(snt.Module):

    def __init__(self, observation_space, action_space, name="QNetwork"):
        super().__init__(name=name)

        self.observation_space = observation_space
        self.action_space = action_space

        # features
        self._torso = snt.nets.MLP(
            [8, 8],
            activation=tf.nn.relu,
            activate_final=True,
            w_init=initializers.he_initializer(),
            name="MLP"
        ) 

        # predictor
        self._q_values = snt.Linear(action_space.n, name="QValues")

    @tf.function
    def __call__(self, obs):
        """Calcula os Q-values de todas as ações para uma dada `obs`."""
        h = self._torso(obs)
        return self._q_values(h)

    @tf.function
    def action_values(self, obs, action):
        """Calcula os Q-values de uma única `action` específica para uma dada `obs`."""
        batch_size = tf.shape(obs)[0]
        indices = tf.stack([tf.range(batch_size, dtype=action.dtype), action], axis=1)
        q_values = tf.gather_nd(self(obs), indices)
        return q_values

    @tf.function
    def hard_update(self, other):
        """Copia os parâmetros da rede `other` para a rede do objeto."""
        for self_var, other_var in zip(self.trainable_variables, other.trainable_variables):
            self_var.assign(other_var)

### Função objetivo (loss)

Lembre-se que o `DQN` se utiliza da regra de atualização baseado em programação dinâmica aproximada do Q-Learning:

$$
\mathcal{L}(\phi) = \mathbb{E}_{(s, a, r, s') \sim \mathcal{D}} [(Q_{\phi}(s, a) - (r + \gamma \max_{a'} Q_{\bar{\phi}}(s', a')))^2]
$$

> **Observação**: lembre que para compor o "alvo" da regressão usamos a rede target $Q_{\bar{\phi}}$. Conforme vimos na aula teórica, o uso de *target networks* é fundamental para melhorar a estabilidade do treinamento. Caso contrário, toda atualização na direção de $\nabla_{\phi} \mathcal{L}(\phi)$ acabaria por alterar também o valor do "alvo" da regressão, tornando o problema de otimização muito mais complicado!

---

**<font color="red">EXERCÍCIO-PROGRAMA 1:</font>**

Implemente a função objetivo do `DQN` conforme exposto acima.

> **Dica**: as funções `tf.reduce_max` e `tf.stop_gradient` podem ser úteis. :)

In [ ]:
def make_q_learning_loss(q_net, target_q_net, gamma=0.99):
    """Recebe a rede online `q_net` e a rede `target_q_net` e devolve o loss function do Q-Learning."""

    @tf.function
    def _loss(batch):
        """Recebe um batch de experiências e devolve o valor da função objetivo para esse batch."""
        obs = batch["obs"]
        actions = batch["action"]
        rewards = batch["reward"]
        next_obs = batch["next_obs"]
        terminals = tf.cast(batch["terminal"], tf.float32)
        
        # SEU CÓDIGO AQUI ===================================

        q_values = ...

        q_targets = ...
    
        # ===================================================
        
        # loss = tf.reduce_mean((q_values - q_targets) ** 2)
        loss = tf.losses.huber(q_values, q_targets)
        return loss

    return _loss

### Atualização (updates)

Uma vez com *loss function* definida, basta instanciar um otimizador escolhendo uma taxa de aprendizado (i.e., `learning_rate`) rodando a célula abaixo:

In [ ]:
def make_update_fn(loss_fn, trainable_variables, learning_rate=1e-3):
    optimizer = snt.optimizers.Adam(learning_rate)

    @tf.function
    def _update_fn(batch):
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            tape.watch(trainable_variables)
            loss = loss_fn(batch)

        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply(grads, trainable_variables)

        grads_and_vars = {var.name: (grad, var) for grad, var in zip(grads, trainable_variables)}

        return loss, grads_and_vars

    return _update_fn

### Política $\epsilon$-*greedy*

O último componente do algoritmo `DQN` é sua política utilizada para explorar. No notebook de hoje, implementaremos a política exploratória mais simples.

Como vimos na aula teórica, a política $\epsilon$-*greedy* escolhe uma ação aleatória com probabilidade $\epsilon$ e escolhe a ação gulosa com probabilidade $1 - \epsilon$.

---

**<font color="red">EXERCÍCIO-PROGRAMA 2:</font>**

Implemente o método especial `EpsilonGreedyPolicy.__call__` para amostar ações de acordo com a definição da política $\epsilon$-*greedy* conforme exposto acima.

> **Dica**: as funções `tf.random.uniform` e `tf.argmax` podem ser úteis. :)

In [ ]:
class EpsilonGreedyPolicy:

    def __init__(self, q_net, start_val=1.0, end_val=0.01, start_step=1_000, end_step=10_000):
        self.q_net = q_net

        self._schedule = schedule.PiecewiseLinearSchedule((start_step, start_val), (end_step, end_val))

        self._step = tf.Variable(0., dtype=tf.float32, name="step")
        self._epsilon = tf.Variable(start_val, dtype=tf.float32, name="epsilon")

    def __call__(self, obs):
        self._epsilon.assign(self._schedule(self._step))
        self._step.assign_add(1)

        batch_size = tf.shape(obs)[0]
        action_dim = self.q_net.action_space.n

        # SEU CÓDIGO AQUI ===================================

        random_actions = ...
        
        greedy_actions = ...
        
        # ===================================================

        return tf.where(
            self._epsilon > tf.random.uniform(shape=(batch_size,)),
            random_actions,
            greedy_actions            
        )

### Agente DQN

Com todos os componentes definidos, estamos finalmente preparados para instanciar um agente `DQN` para o ambiente `CartPole-v1`. Execute a célula abaixo para criar a classe `DQN`.

In [ ]:
class DQN:

    def __init__(
        self,
        observation_space,
        action_space,
        gamma=0.99,
        target_update_freq=1000,
        learning_rate=1e-3, 
        checkpoint_dir="ckpt"
    ):
        self.observation_space = observation_space
        self.action_space = action_space

        self.gamma = gamma
        self.target_update_freq = target_update_freq
        self.learning_rate = learning_rate

        self.q_net = QNetwork(self.observation_space, self.action_space, name="QNet")
        self.target_q_net = QNetwork(self.observation_space, self.action_space, name="TargetQNet")

        self.policy = EpsilonGreedyPolicy(self.q_net)
        
        self._ckpt_dir = checkpoint_dir
        self._ckpt = tf.train.Checkpoint(q_net=self.q_net)
        self._ckpt_manager = tf.train.CheckpointManager(self._ckpt, directory=self._ckpt_dir, max_to_keep=1)

        self._step = tf.Variable(0, dtype=tf.int32, name="step")

    def build(self):
        """Cria as variáveis das redes online e target e sincroniza inicialmente."""
        input_spec = tf.TensorSpec(self.observation_space.shape, dtype=tf.float32)
        tf_utils.create_variables(self.q_net, input_spec)
        tf_utils.create_variables(self.target_q_net, input_spec)
        self.target_q_net.hard_update(self.q_net)

    def compile(self):
        """Compila a DQN loss junto com a Q-network."""
        self.update_learner = make_update_fn(
            make_q_learning_loss(self.q_net, self.target_q_net, gamma=self.gamma),
            self.q_net.trainable_variables,
            learning_rate=self.learning_rate
        )

    def step(self, obs, training=True):
        """Escolhe a ação para a observação dada."""
        obs = tf.convert_to_tensor(obs, dtype=tf.float32)
        action = self.policy(obs) if training else tf.argmax(self.q_net(obs), axis=-1)
        return action.numpy()

    def learn(self, batch):
        """Recebe um batch de experiências, atualiza os parâmetros das redes, e devolve algumas métricas."""
        # atualiza q_net
        loss, grads_and_vars = self.update_learner(batch)

        # sincroniza target_q_net
        self._step.assign_add(1)
        if self._step % self.target_update_freq == 0:
            self.target_q_net.hard_update(self.q_net)

        # métricas de monitoramento
        stats = {
            "loss": loss,
            "epsilon": self.policy._epsilon,
            "vars": {key: variable for key, (_, variable) in grads_and_vars.items()},
            "grads": {f"grad_{key}": grad for key, (grad, _) in grads_and_vars.items()},
        }

        return stats

    def save(self):
        """Salva o estado atual do agente (i.e., o valor dos parâmetros da rede online) nesse momento."""
        return self._ckpt_manager.save()

    def restore(self, save_path=None):
        """Carrega o último checkpoint salvo anteriormente no `save_path`."""
        if not save_path:
            save_path = self._ckpt_manager.latest_checkpoint
        return self._ckpt.restore(save_path)

## Protocolo de treinamento, avaliação e teste

Com a classe do `DQN` definida é hora de treinar o agente e avaliá-lo. Faremos isso seguindo um protocolo de treinamento e avaliação definido pelas funções `train` e  `evaluate` abaixo.

<img src="img/rl-training.png" alt="Agent-Env Loop" style="width: 850px;"/>

Tente entender como os hiperparâmetros de início de treinamento `learning_starts` e frequência de atualizações `learn_every` e avaliação `evaluation_freq` definem o protocolo.

> **Observação**: note que embora o protocolo abaixo seja bastante genérico, tenha em mente que diferentes trabalhos alteram a maneira como os processos de coleta de dados, aprendizado e avaliação se intercalam.

In [ ]:
def train(
    agent,
    env,
    test_env,
    replay,
    logger,
    total_timesteps=20_000,
    learning_starts=1_000,
    learn_every=1,
    evaluation_freq=1_000
):  
    timesteps = 0
    episodes = 0
    episode_returns = deque(maxlen=100)

    best_episode_reward_mean = -np.inf
    
    with trange(total_timesteps, desc="training") as pbar:

        while timesteps < total_timesteps:
            obs = env.reset()
            episode_return = 0.0

            for episode_length in range(1, env.spec.max_episode_steps + 1):

                # collect
                action = agent.step(np.expand_dims(obs, axis=0), training=True)[0]
                next_obs, reward, done, info = env.step(action)

                timesteps += 1
                episode_return += reward

                # add experience to replay buffer
                terminal = done if episode_length < env.spec.max_episode_steps else False
                replay.add(obs, action, reward, terminal, next_obs)

                # training
                if timesteps >= learning_starts and timesteps % learn_every == 0:
                    batch = replay.sample()
                    stats = agent.learn(batch)
                    stats["episode_return_mean"] = np.mean(episode_returns)
                    logger.log(timesteps, stats, label="train") # logging

                # evaluation
                if timesteps % evaluation_freq == 0:
                    stats = evaluate(agent, test_env)
                    logger.log(timesteps, stats, label="evaluation") # logging

                    # checkpointing
                    if stats["episode_return_mean"] > best_episode_reward_mean:
                        agent.save()
                        best_episode_reward_mean = stats["episode_return_mean"]

                if done:
                    break

                obs = next_obs

            episodes += 1
            episode_returns.append(episode_return)

            # logging
            stats = {
                "episodes": episodes,
                "episode_length": episode_length,
                "episode_return": episode_return,
            }
            logger.log(timesteps, stats, label="collect")
            logger.flush()

            pbar.update(episode_length)
            pbar.set_postfix(timesteps=timesteps, episodes=episodes, avg_returns=np.mean(episode_returns) if episode_returns else None)

    # final evaluation
    stats = evaluate(agent, test_env)
    logger.log(timesteps, stats, label="evaluation")
    logger.flush()

Para avaliarmos o agente, utilizaremos o `eval_env` que foi criado como um ambiente paralelizado (contendo `env.num_envs` rodando de forma assíncrona em paralelo). 

> **Observação**: Note no código abaixo, como esse tipo de ambiente altera ligeiramente o ciclo de interação agente-ambiente que vimos nas últimas aulas. Para maiores detalhes, consulte a documentação de `gym.vector.make` e o código dos módulos em [https://github.com/openai/gym/tree/master/gym/vector](https://github.com/openai/gym/tree/master/gym/vector).

In [ ]:
def evaluate(agent, env):
    total_reward = np.zeros((env.num_envs,))
    episode_length = np.zeros((env.num_envs,))

    obs = env.reset()
    dones = np.array([False] * env.num_envs)

    while not np.all(dones):
        action = agent.step(obs, training=False)
        obs, reward, done, _ = env.step(action)
        total_reward += (1 - dones) * reward
        episode_length += (1 - dones)
        dones = np.logical_or(dones, done)

    return {
        "episode_return_mean": np.mean(total_reward),
        "episode_return_min": np.min(total_reward),
        "episode_return_max": np.max(total_reward),
    } 

Execute a célula abaixo para definir um ciclo de interação agente-ambiente para renderizar episódios do agente após o treinamento e então verificar qualitativamente quão boa foi a política que o agente aprendeu.

In [ ]:
def test(agent, env, episodes=3, wait=None):
    for episode in range(episodes):
        obs = env.reset()
        env.render()
        done = False

        while not done:
            action = agent.step(np.expand_dims(obs, axis=0), training=False)[0]
            obs, reward, done, _ = env.step(action)
            env.render()
            if wait:
                time.sleep(wait)

    env.close()

## Treinando DQN no CartPole-v1

Finalmente, temos todo o código necessário para treinarmos o `DQN` no `CartPole-v1`. Antes de iniciarmos o treinamento, execute a célula abaixo para instanciarmos o `tensorboard`, a ferramenta de *logging* e monitoramento do TensorFlow. Consulte a documentação e os tutoriais disponíveis em [https://www.tensorflow.org/tensorboard](https://www.tensorflow.org/tensorboard) para maiores informações.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs --reload_interval 10

In [ ]:
def run(env, total_timesteps=20_000, trials=3):
    for _ in range(trials):
        timestamp = datetime.now().strftime("%Y-%m-%d-%H:%M")
        run_id = osp.join(f"dqn-{env.spec.id}".lower(), timestamp)

        logger = logging.TFLogger(run_id, base_dir="logs")

        buffer = replay.ReplayBuffer(env.observation_space, env.action_space, max_size=total_timesteps, batch_size=64)
        buffer.build()

        agent = DQN(env.observation_space, env.action_space, checkpoint_dir=f"ckpt/{run_id}")
        agent.build()
        agent.compile()

        train(agent, env, eval_env, buffer, logger, total_timesteps=total_timesteps)

In [ ]:
run(env)

Agora é só buscar um café esperar o resultado do treinamento. ;)

### Teste do Agente no CartPole-v1

Escolha um dos agentes treinados acima para testar e visualizar seu comportamento com o código abaixo:

In [ ]:
checkpoint_dir = "ckpt/dqn-cartpole-v1/2021-01-28-01:49" # altere essa linha para escolher qual checkpoint do agente

agent = DQN(env.observation_space, env.action_space, checkpoint_dir=checkpoint_dir)
agent.build()
agent.restore()

test(agent, test_env)

## DQN no Atari (Pong)

Nesta 2a parte do notebook, implementaremos algumas melhorias no agente do `DQN` para treinarmos um agente para o ambiente do `PongNoFrameskip-v4`.

### Ambiente - PongNoFrameskip-v4

Para treinar o `DQN` na versão original do artigo (Mnih et al, 2015) para os jogos do Atari, precisamos aplicar algumas transformações em cima do ambiente nativo do simulare ALE encapsulado via pacote `gym`. Note o uso dos `gym.wrappers` na construção do ambiente no código abaixo.

> **Observação**: o aluno interessado em resolver outros jogos do Atari deve consular o artigo [Revisiting the Arcade Learning Environment: Evaluation Protocols and Open Problems for General Agents](https://www.jair.org/index.php/jair/article/download/11182/26388) para maiores detalhes sobre o uso de `wrappers` e transformações do ambiente.

In [ ]:
def make_envs(env_id):
    assert "NoFrameskip" in env_id

    def _wrapper(env, max_episode_steps=None, num_stack=4, terminal_on_life_loss=False):
        # limita o número máximo de passos de interação em um episódio
        env = TimeLimit(env, max_episode_steps=max_episode_steps)
        
        # diminue o tamanho da imagem, muda de RGB para greyscale, adiciona action repeats e frame_skip
        env = AtariPreprocessing(env=env, frame_skip=4, terminal_on_life_loss=terminal_on_life_loss)
        
        # concatena frames consecutimos como observação -- importante para tentar aproximar a propriedade de Markov do estado
        env = FrameStack(env=env, num_stack=num_stack)
        return env

    env = _wrapper(gym.make(env_id), max_episode_steps=50_000, terminal_on_life_loss=True)
    eval_env = gym.vector.make(env_id, num_envs=10, asynchronous=True, wrappers=lambda env: _wrapper(env, max_episode_steps=108_000))
    test_env = _wrapper(gym.make(env_id), max_episode_steps=108_000)

    return env, eval_env, test_env

In [ ]:
env, eval_env, test_env = make_envs("PongNoFrameskip-v4")

### Double Q-Learning

É comum um agente baseado no algoritmo do `DQN` superestimar a função ao longo do tempo de treinamento, mesmo com o uso de *target networks*. Diferentemente do ambiente do `CartPole-v1`, em jogos de Atari é preciso centenas de milhares ou mesmo milhões de `timesteps` para convergir um agente `DQN`. Dessa forma esse problema de super-estimação dos valores de $Q(s, a)$ pode contribuir para instabilidade e consequente divergência do treinamento.

O problema de super-estimação é decorrente da aproximação que fazemos no Q-Learning:
$$
\mathbb{E}_{s' \sim p(\cdot|s, a)} [\max_{a'} \tilde{Q}(s', a')] \approx \max_{a'} \tilde{Q}(s', a')~,
$$
onde usamos apenas uma única amostra para estimar o valor esperado.

**Observação**: para o aluno interessado consulte o artigo [Deep reinforcement learning with double q-learning](https://ojs.aaai.org/index.php/AAAI/article/download/10295/10154).



A solução encontrada se chama *Double Q-Learning* na qual alteramos ligeiramente a maneira como calculamos os Q-values para o *target* da regressão:

$$
\mathcal{L}(\phi) = \mathbb{E}_{(s, a, r, s') \sim \mathcal{D}} [(Q_{\phi}(s, a) - (r + \gamma Q_{\bar{\phi}}(s', \arg\max_{a'} Q_{\phi}(s', a'))))^2]
$$

> **Observação**: note que a única alteração é usar a rede "online" $Q_\phi$ para escolher a melhor ação no próximo estado $s'$, mas continuar a avaliar com a rede "target" $Q_{\bar{\phi}}$ .


---

**<font color="red">EXERCÍCIO-PROGRAMA 3:</font>**

Termine a implementação da função  `make_double_q_learning_loss` para calcular a função objetivo do Double Q-Learning.

> **Dica**: as funções `tf.reduce_max` e `tf.stop_gradient` podem ser úteis. :)

In [ ]:
def make_double_q_learning_loss(q_net, target_q_net, gamma=0.99):
    """Recebe a rede online `q_net` e a rede `target_q_net` e devolve o loss function do Double Q-Learning."""

    @tf.function
    def _loss(batch):
        """Recebe um batch de experiências e devolve o valor da função objetivo para esse batch."""
        obs = batch["obs"]
        actions = batch["action"]
        rewards = tf.clip_by_value(batch["reward"], -1., 1.)
        next_obs = batch["next_obs"]
        terminals = tf.cast(batch["terminal"], tf.float32)

        # SEU CÓDIGO AQUI ===================================

        q_values = ...

        q_targets = ...
    
        # ===================================================

        # loss = tf.reduce_mean((q_values - q_targets) ** 2)
        loss = tf.losses.huber(q_values, q_targets)
        return loss

    return _loss

### Dueling QNetwork

A última modificação do `DQN` que precisamos são as chamadas *Dueling Networks*. A ideia básica é introduzir estrutura na rede neural que tenta predizer os valores de $Q(s, a)$.

Para isso definimos a chamada *Advantage function* (não vista na aula teórica ainda):
$$
A^{\pi}(s, a) = Q^\pi(s, a) - V^\pi(s)
$$

A função *advantage* estima o quão melhor é uma ação com relação ao valor médio sobre todas as ações (lembre-se que $V^\pi(s) = \mathbb{E}_{a \sim \pi(\cdot|s)} Q^\pi(s, a)$).

Note que $\mathbb{E}_{a \sim \pi(\cdot|s)} A^{\pi}(s, a) = \mathbb{E}_{a \sim \pi(\cdot|s)} [Q^\pi(s, a) - V^\pi(s)] = \mathbb{E}_{a \sim \pi(\cdot|s)} [Q^\pi(s, a)] - V^\pi(s) = V^\pi(s) - V^\pi(s) = 0$, isto é, como esperado a função vantagem tem média zero sobre as ações!

Dessa forma, podemos decompor $Q(s, a)$ como a soma de uma componente de média zero com a função Valor do estado:
$$
Q^\pi(s, a)  = A^{\pi}(s, a) + V^\pi(s)
$$

<img src="img/dueling-q-net.png" alt="Agent-Env Loop" style="width: 650px;"/>

Na prática, essa estrutura pode facilar o aprendizado da função para problemas em que em certas situações é mais fácil prever diferenças entre ações do que estimar o retorno propriamente dita!

> **Observação**: o aluno interessado pode consular o artigo [Dueling network architectures for deep reinforcement learning](http://proceedings.mlr.press/v48/wangf16.pdf) para maiores detalhes.

---

**<font color="red">EXERCÍCIO-PROGRAMA 4:</font>**

Termine a implementação do método especial `DuelingQNetwork.__call__` usada para estimar os valores de $Q(s, \cdot)$.

> **Dica**: a função `tf.reduce_mean` pode ser útil. :)

In [ ]:
class DuelingQNetwork(snt.Module):

    def __init__(self, observation_space, action_space, name="AtariQNetwork"):
        super().__init__(name=name)

        self.observation_space = observation_space
        self.action_space = action_space

        # Atari torso
        self._torso = snt.Sequential([
            snt.Conv2D(32, kernel_shape=8, stride=4, padding="VALID", w_init=initializers.he_initializer(), name="Conv1"),
            tf.nn.relu,
            snt.Conv2D(64, kernel_shape=4, stride=2, padding="VALID", w_init=initializers.he_initializer(), name="Conv2"),
            tf.nn.relu,
            snt.Conv2D(64, kernel_shape=3, stride=1, padding="VALID", w_init=initializers.he_initializer(), name="Conv3"),
            tf.nn.relu,
            snt.Flatten(),
        ])

        # predictors (dueling network)
        self._value_mlp = snt.nets.MLP([512, 1], w_init=initializers.he_initializer(), activation=tf.nn.relu, activate_final=False, name="Value")
        self._advantage_mlp = snt.nets.MLP([512, action_space.n], w_init=initializers.he_initializer(), activation=tf.nn.relu, activate_final=False, name="Advantage")

    @tf.function
    def __call__(self, obs):
         """Calcula os Q-values de todas as ações para uma dada `obs`."""

        # pre-processamento
        obs = tf.cast(obs, dtype=tf.float32) / 255.
        obs = tf.transpose(obs, perm=[0, 2, 3, 1])

        # features
        h = self._torso(obs)

        # predições

        # SEU CÓDIGO AQUI ===================================

        q_values = ...

        # ===================================================

        return q_values

    @tf.function
    def action_values(self, obs, action):
        """Calcula os Q-values de uma única `action` específica para uma dada `obs`."""
        batch_size = tf.shape(obs)[0]
        indices = tf.stack([tf.range(batch_size, dtype=action.dtype), action], axis=1)
        q_values = tf.gather_nd(self(obs), indices)
        return q_values

    @tf.function
    def hard_update(self, other):
        """Copia os parâmetros da rede `other` para a rede do objeto."""
        for self_var, other_var in zip(self.trainable_variables, other.trainable_variables):
            self_var.assign(other_var)

### Agente Double DQN (DDQN)

Agora é só juntar os novos componentes no agente `DDQN`.

In [ ]:
class DDQN:

    def __init__(
        self,
        observation_space,
        action_space,
        gamma=0.99,
        target_update_freq=1000,
        learning_rate=2.5e-4,
        checkpoint_dir="ckpt"
    ):
        self.observation_space = observation_space
        self.action_space = action_space

        self.gamma = gamma
        self.target_update_freq = target_update_freq
        self.learning_rate = learning_rate

        self.q_net = DuelingQNetwork(self.observation_space, self.action_space, name="QNet")
        self.target_q_net = DuelingQNetwork(self.observation_space, self.action_space, name="TargetQNet")

        self.policy = EpsilonGreedyPolicy(self.q_net, start_val=1.0, end_val=0.01, start_step=10_000, end_step=250_000)

        self._ckpt_dir = checkpoint_dir
        self._ckpt = tf.train.Checkpoint(q_net=self.q_net)
        self._ckpt_manager = tf.train.CheckpointManager(self._ckpt, directory=self._ckpt_dir, max_to_keep=1)

        self._step = tf.Variable(0, dtype=tf.int32, name="step")

    def build(self):
        """Cria as variáveis das redes online e target e sincroniza inicialmente."""
        input_spec = tf.TensorSpec(self.observation_space.shape, dtype=tf.float32)
        tf_utils.create_variables(self.q_net, input_spec)
        tf_utils.create_variables(self.target_q_net, input_spec)
        self.target_q_net.hard_update(self.q_net)

    def compile(self):
        """Compila a Double DQN loss junto com a DuelingQNetwork."""
        self.update_learner = make_update_fn(
            make_double_q_learning_loss(self.q_net, self.target_q_net, gamma=self.gamma),
            self.q_net.trainable_variables,
            learning_rate=self.learning_rate
        )

    def step(self, obs, training=True):
        """Escolhe a ação para a observação dada."""
        obs = tf.convert_to_tensor(obs, dtype=tf.float32)
        action = self.policy(obs) if training else tf.argmax(self.q_net(obs), axis=-1)
        return action.numpy()

    def learn(self, batch):
        """Recebe um batch de experiências, atualiza os parâmetros das redes, e devolve algumas métricas."""
        loss, grads_and_vars = self.update_learner(batch)

        # update target network
        self._step.assign_add(1)
        if self._step % self.target_update_freq == 0:
            self.target_q_net.hard_update(self.q_net)

        stats = {
            "loss": loss,
            #"q_values_mean": tf.reduce_mean(self.q_net(batch["obs"])),
            "epsilon": self.policy._epsilon,
            "vars": {key: variable for key, (_, variable) in grads_and_vars.items()},
            "grads": {f"grad_{key}": grad for key, (grad, _) in grads_and_vars.items()},
        }

        return stats

    def save(self):
        """Salva o estado atual do agente (i.e., o valor dos parâmetros da rede online) nesse momento."""
        return self._ckpt_manager.save()

    def restore(self, save_path=None):
        """Carrega o último checkpoint salvo anteriormente no `save_path`."""
        if not save_path:
            save_path = self._ckpt_manager.latest_checkpoint
        return self._ckpt.restore(save_path)

### Treinando DDQN no Pong (do zero)

Execute a célula abaixo para definir o protocolo de treinamento e avaliação próprio para jogos de Atari.

In [ ]:
def train(
    agent, 
    env, 
    test_env, 
    replay,
    logger,
    total_timesteps=500_000, 
    learning_starts=2_500, 
    learn_every=1, 
    evaluation_freq=5_000
):
    timesteps = 0
    episodes = 0
    episode_returns = deque(maxlen=20)

    while timesteps < total_timesteps:

        obs = env.reset()
        episode_return = 0.0

        for episode_length in range(1, env.spec.max_episode_steps + 1):

            # collect
            action = agent.step(np.expand_dims(obs, axis=0), training=True)[0]
            next_obs, reward, done, info = env.step(action)

            timesteps += 1
            episode_return += reward

            # add experience to replay buffer
            terminal = done if episode_length < env.spec.max_episode_steps else False
            replay.add(obs, action, reward, terminal, next_obs)

            # training
            if timesteps >= learning_starts and timesteps % learn_every == 0:
                batch = replay.sample()
                train_stats = agent.learn(batch)

            # evaluation
            if timesteps % evaluation_freq == 0:
                eval_stats = evaluate(agent, eval_env)

                # logging
                train_stats["episode_reward_mean"] = np.mean(episode_returns)
                logger.log(timesteps, train_stats, label="train")
                logger.log(timesteps, eval_stats, label="evaluation")

                # checkpointing
                agent.save()

            if done:
                break

            obs = next_obs

        episodes += 1
        episode_returns.append(episode_return)

        # logging
        stats = {
            "episodes": episodes,
            "episode_length": episode_length,
            "episode_return": episode_return,
        }
        logger.log(timesteps, stats, label="collect")
        logger.flush()

        print(f"Timesteps = {timesteps:5d} | Episodes = {episodes:4d} | Episode Length = {episode_length:4d} | Episode Return = {episode_return:.3f}")

    # final evaluation
    stats = evaluate(agent, eval_env)
    logger.log(timesteps, stats, label="evaluation")
    logger.flush()

    agent.save()

Na célula abaixo, criamos o `AtariReplayBuffer` e o agente `DDQN`.

> **IMPORTANTE**: para armazenar 500K timesteps no buffer será necessário aproximadamente 4G de memória RAM. Se você tiver limitações de memória diminua para 200K ou menos.

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d-%H:%M")
run_id = osp.join(f"ddqn-{env.spec.id}".lower(), timestamp)

logger = logging.TFLogger(run_id, base_dir="logs")

buffer = replay.AtariReplayBuffer(env.observation_space, env.action_space, max_size=500_000, batch_size=32)
buffer.build()

agent = DDQN(env.observation_space, env.action_space, checkpoint_dir=f"ckpt/{run_id}")
agent.build()
agent.compile()

> **Observação**: se você quiser treinar o agente do zero, isto é, extrator de features da rede convolucional e o preditor em cima das features, execute o código abaixo. O treinamento para 500K timesteps deve demorar algumas horas dependendo do seu hardware. Tipicamente, em GPUs deve ser necessário algo entre 1h e 3h. Para treinamento exclusivamente em CPUs, o treinamento deve demorar algo em torno de mais de 12h. Fique atento à isso, pois o seu computador pode super-aquecer se você não estiver preparado para deixar o computador rodando por tanto tempo!

In [ ]:
train(agent, env, eval_env, buffer, logger, total_timesteps=500_000, learning_starts=2_500, evaluation_freq=5_000)

### Treinando DDQN no Pong (com features pré-treinadas)

Se você quiser treinar apenas as camadas de predição da `DuelingQNetwork`, pode reaproveitar o extrator de features do agente pré-treinado disponível em `ckpt/ddqn-pongnoframeskip-v4/2021-01-19-16:30/ckpt-101`. Isso deve provavelmente acelerar o treinamento em CPUs em algumas horas.

Execute as próximas células.

In [ ]:
def load_pre_trained_agent(run_id, checkpoint_dir, features_only=True):
    trained_agent = DDQN(env.observation_space, env.action_space, checkpoint_dir=f"ckpt/{run_id}")
    trained_agent.build()
    trained_agent.restore(checkpoint_dir).assert_consumed()

    if features_only:
        # transfer learning
        trainable_variables = []
        for online_var, target_var in zip(trained_agent.q_net.trainable_variables, trained_agent.target_q_net.trainable_variables):
            if "Conv" in online_var.name: # Conv layers
                target_var.assign(online_var)
            else: # MLP layers
                online_var.assign(target_var)
                trainable_variables.append(online_var)

        trained_agent.update_learner = make_update_fn(
            make_double_q_learning_loss(trained_agent.q_net, trained_agent.target_q_net, gamma=trained_agent.gamma),
            trainable_variables,
            learning_rate=trained_agent.learning_rate
        )

    return trained_agent

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d-%H:%M")
run_id = osp.join(f"ddqn-{env.spec.id}".lower(), timestamp)

logger = logging.TFLogger(run_id, base_dir="logs")

buffer = replay.AtariReplayBuffer(env.observation_space, env.action_space, max_size=500_000, batch_size=32)
buffer.build()

checkpoint_dir = "ckpt/ddqn-pongnoframeskip-v4/2021-01-19-16:30/ckpt-101"
agent = load_pre_trained_agent(run_id, checkpoint_dir, features_only=True)

In [ ]:
train(agent, env, eval_env, buffer, logger, total_timesteps=500_000, learning_starts=2_500, evaluation_freq=5_000)

### Teste do Agente no PongNoFrameskip-v4

Por fim, escolha o agente `DDQN` treinado acima para testar e visualizar seu comportamento com o código abaixo:

In [ ]:
checkpoint_dir = "ckpt/ddqn-pongnoframeskip-v4/2021-01-28-07:18" # altere essa linha para escolher qual checkpoint do agente

agent = DDQN(env.observation_space, env.action_space, checkpoint_dir=checkpoint_dir)
agent.build()
agent.restore()

test(agent, test_env)